### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
print('The User-Movie matrix has {} users and {} movies.'.format(ratings_mat.shape[0], ratings_mat.shape[1]))

The User-Movie matrix has 20 users and 4 movies.


In [3]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users =     ratings_mat.shape[0]    # number of rows in the matrix
    n_movies =    ratings_mat.shape[1]    # number of columns in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))      # total number of non-NaN ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat =  np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for i in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # for each user-movie pair
        for user in range(n_users):
            for movie in range(n_movies):
                
                # if the rating exists
                if ratings_mat[user, movie] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    user_row = user_mat[user, :]
                    movie_col = movie_mat[:, movie]
                    error = ratings_mat[user, movie] - np.dot(user_row, movie_col)

                    # keep track of the total sum of squared errors for the matrix
                    sse_accum += error**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for latent in range(latent_features):
                        user_mat[user, latent]   += learning_rate * (2 * error * movie_mat[latent, movie])
                        movie_mat[latent, movie] += learning_rate * (2 * error * user_mat[user, latent])

        # print results for iteration
        #print('User matrix:\n\n{}\n\nMovie matrix:\n\n{}\n\n'.format(user_mat, movie_mat))
        print("%d \t\t %f" % (i + 1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [4]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 41.506846
2 		 12.967073
3 		 3.846694
4 		 2.825559
5 		 2.697404
6 		 2.655893
7 		 2.630218
8 		 2.606249
9 		 2.580198
10 		 2.550728


In [5]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.95580169   8.94647742  10.79817187  10.03259719]
 [  8.79999651   7.41713357   9.12809671   8.64156585]
 [  7.53051527   7.12854225   8.34265272   7.84411121]
 [  9.7856166    8.08755837  10.03964347   9.53637937]
 [  8.58000014   7.33559273   8.96549038   8.54848435]
 [  6.94066011   5.67541289   7.58128631   6.64106529]
 [  9.14084055   8.28081935   9.38902414   9.3498345 ]
 [  8.28180828   7.24321721   8.43659895   8.27053307]
 [  8.45580469   7.38854703   8.99463584   8.43078414]
 [  7.9278764    6.33665356   8.16052989   7.66353655]
 [  8.98689899   7.71976531   9.32083859   8.84885411]
 [  9.50764462   8.52904367   9.95828178   9.6669863 ]
 [  9.36787989   8.11570898   9.67573017   9.29341047]
 [  6.39158229   5.94793866   6.98722089   6.63563927]
 [  9.73253382   8.55047672  10.19968348   9.74115796]
 [  9.68253688   8.54275528  10.31651656   9.68790922]
 [  8.28101673   7.26480885   8.6477373    8.36083351]
 [  7.39420738   6.12306772   7.22514579   7.18736655]
 [  7.9921

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [6]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.758586
2 		 14.684537
3 		 3.975715
4 		 2.801856
5 		 2.695910
6 		 2.663867
7 		 2.640857
8 		 2.616800
9 		 2.589446
10 		 2.558069
11 		 2.522141
12 		 2.481119
13 		 2.434445
14 		 2.381588
15 		 2.322090
16 		 2.255612
17 		 2.181984
18 		 2.101252
19 		 2.013722
20 		 1.919975
21 		 1.820878
22 		 1.717545
23 		 1.611290
24 		 1.503542
25 		 1.395754
26 		 1.289317
27 		 1.185484
28 		 1.085326
29 		 0.989706
30 		 0.899278
31 		 0.814500
32 		 0.735651
33 		 0.662851
34 		 0.596076
35 		 0.535180
36 		 0.479914
37 		 0.429949
38 		 0.384901
39 		 0.344357
40 		 0.307897
41 		 0.275116
42 		 0.245635
43 		 0.219108
44 		 0.195232
45 		 0.173740
46 		 0.154401
47 		 0.137011
48 		 0.121395
49 		 0.107395
50 		 0.094870
51 		 0.083690
52 		 0.073734
53 		 0.064891
54 		 0.057055
55 		 0.050127
56 		 0.044016
57 		 0.038635
58 		 0.033905
59 		 0.029753
60 		 0.026113
61 		 0.022923
62 		 0.020130
63 		 0.017686
64 		

In [7]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99999917   9.99999949   9.99999985  10.00000091]
 [ 10.00000103   4.00000077   9.00000029   9.99999876]
 [  7.99999977   8.9999999    9.99999998   5.00000023]
 [  9.00000123   8.00000099  10.00000035   9.99999845]
 [  9.99999711   4.99999798   8.99999929   9.00000335]
 [  6.00000088   4.00000071  10.00000026   5.9999989 ]
 [  9.00000261   8.00000191  10.00000068   8.99999688]
 [ 10.00000174   5.00000127   9.00000042   7.99999789]
 [  7.00000378   8.00000263  10.00000088   7.99999552]
 [  8.99999943   4.99999946   8.99999975   7.00000069]
 [  9.00000207   8.00000139  10.00000041   7.99999753]
 [  8.99999836   9.99999872   9.99999947   9.00000188]
 [  9.99999772   8.99999836   9.99999938   8.00000262]
 [  5.00000274   8.00000195   5.00000066   7.99999672]
 [ 10.0000047    8.00000323  10.00000107   9.99999445]
 [  8.99999782   8.99999836   9.99999931  10.00000247]
 [  9.00000227   8.00000156   8.00000047   7.99999727]
 [  9.99999882   7.99999912   0.99999963  10.00000131]
 [  4.9999

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [8]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [9]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 44.141988
2 		 15.915789
3 		 4.559246
4 		 2.940151
5 		 2.713885
6 		 2.642205
7 		 2.599136
8 		 2.560314
9 		 2.519217
10 		 2.473597
11 		 2.422322
12 		 2.364597
13 		 2.299781
14 		 2.227371
15 		 2.147023
16 		 2.058607
17 		 1.962250
18 		 1.858380
19 		 1.747754
20 		 1.631466
21 		 1.510917
22 		 1.387766
23 		 1.263845
24 		 1.141068
25 		 1.021315
26 		 0.906333
27 		 0.797641
28 		 0.696464
29 		 0.603681
30 		 0.519811
31 		 0.445023
32 		 0.379171
33 		 0.321848
34 		 0.272449
35 		 0.230245
36 		 0.194441
37 		 0.164232
38 		 0.138843
39 		 0.117559
40 		 0.099738
41 		 0.084817
42 		 0.072315
43 		 0.061822
44 		 0.052997
45 		 0.045555
46 		 0.039263
47 		 0.033927
48 		 0.029388
49 		 0.025515
50 		 0.022201
51 		 0.019357
52 		 0.016909
53 		 0.014797
54 		 0.012970
55 		 0.011386
56 		 0.010010
57 		 0.008812
58 		 0.007766
59 		 0.006853
60 		 0.006053
61 		 0.005351
62 		 0.004735
63 		 0.004194
64 		

In [10]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.834692242696505:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [11]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.403308
2 		 10.745294
3 		 7.367827
4 		 5.688516
5 		 4.631897
6 		 3.892397
7 		 3.341260
8 		 2.913415
9 		 2.571849
10 		 2.293535
11 		 2.063181
12 		 1.870150
13 		 1.706792
14 		 1.567454
15 		 1.447860
16 		 1.344697
17 		 1.255333
18 		 1.177632
19 		 1.109823
20 		 1.050421


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [12]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()
# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0] * first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [13]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
